In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# https://pythonhosted.org/pywebhdfs/

In [2]:
import pyspark
from pyspark.ml.feature import Word2Vec
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.ml import Pipeline, PipelineModel
import os
 


In [3]:
os.environ['PYSPARK_PYTHON'] = '/home/hduser/anaconda3/bin/python'
# The value is the python command of the version required to start the master and worker in the Linux system
os.environ['PYSPARK_DRIVER_PYTHON'] = r"C:\Users\billy\anaconda3\python.exe"
# The value is the spark directory in the local windows system
os.environ['SPARK_HOME'] = 'C:/spark'
# The value is the local IP, and the IP required to establish a connection, to prevent connection failure when multiple network cards
#os.environ['SPARK_LOCAL_IP'] = '192.168.56.1'
os.environ['HADOOP_HOME'] = "D:/hadoop-3.3.0"
os.environ['HADOOP_CONF_DIR'] = "/usr/local/hadoop/etc/hadoop"

In [4]:
conf  = pyspark.SparkConf().setAppName('sql').setMaster('spark://192.168.133.4:7077').set(
    "spark.submit.deployMode","client").set('spark.driver.memory','2g').set(
        'spark.executor.memory', '2g').set('spark.executor.cores', 1).set(
        'spark.network.timeout', 600).set('spark.executor.heartbeatInterval', 120).set(
    'spark.cores.max', 4).set("spark.driver.host","192.168.133.1").set("spark.driver.port","9999")#.set('spark.python.profile','true')
sc = pyspark.SparkContext(conf=conf)

In [5]:
path = "hdfs://192.168.133.4:9000/user/hduser/"

In [6]:
RawUserRDD = sc.textFile(path+"data/u.user")

In [7]:
RawUserRDD.first()

'1|24|M|technician|85711'

In [8]:
RawUserRDD.take(5)

['1|24|M|technician|85711',
 '2|53|F|other|94043',
 '3|23|M|writer|32067',
 '4|24|M|technician|43537',
 '5|33|F|other|15213']

In [9]:
userRDD = RawUserRDD.map(lambda x:x.split('|'))

In [10]:
sqlContext = SparkSession.builder.getOrCreate()

In [11]:
from pyspark.sql import Row
user_Rows = userRDD.map(lambda p:
                       Row(
                         userid = int(p[0]),
                            age = int(p[1]),
                           gender=p[2],
                           occupation=p[3],
                           zipcode=p[4]
                       )
            )
user_Rows.take(5)

[Row(userid=1, age=24, gender='M', occupation='technician', zipcode='85711'),
 Row(userid=2, age=53, gender='F', occupation='other', zipcode='94043'),
 Row(userid=3, age=23, gender='M', occupation='writer', zipcode='32067'),
 Row(userid=4, age=24, gender='M', occupation='technician', zipcode='43537'),
 Row(userid=5, age=33, gender='F', occupation='other', zipcode='15213')]

In [12]:
user_df = sqlContext.createDataFrame(user_Rows)
user_df.printSchema()

root
 |-- userid: long (nullable = true)
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- zipcode: string (nullable = true)



In [13]:
user_df.registerTempTable("user_table")

In [14]:
user_df.approxQuantile("userid", [0.25,0.5,0.7], 0)

[236.0, 472.0, 661.0]

In [15]:
sqlContext.sql("""SELECT * FROM user_table""").show()

+------+---+------+-------------+-------+
|userid|age|gender|   occupation|zipcode|
+------+---+------+-------------+-------+
|     1| 24|     M|   technician|  85711|
|     2| 53|     F|        other|  94043|
|     3| 23|     M|       writer|  32067|
|     4| 24|     M|   technician|  43537|
|     5| 33|     F|        other|  15213|
|     6| 42|     M|    executive|  98101|
|     7| 57|     M|administrator|  91344|
|     8| 36|     M|administrator|  05201|
|     9| 29|     M|      student|  01002|
|    10| 53|     M|       lawyer|  90703|
|    11| 39|     F|        other|  30329|
|    12| 28|     F|        other|  06405|
|    13| 47|     M|     educator|  29206|
|    14| 45|     M|    scientist|  55106|
|    15| 49|     F|     educator|  97301|
|    16| 21|     M|entertainment|  10309|
|    17| 30|     M|   programmer|  06355|
|    18| 35|     F|        other|  37212|
|    19| 40|     M|    librarian|  02138|
|    20| 42|     F|    homemaker|  95660|
+------+---+------+-------------+-

In [16]:
sqlContext.sql("""SELECT count(*) as counts FROM user_table""").show()

+------+
|counts|
+------+
|   943|
+------+



In [17]:
sqlContext.sql("""SELECT * FROM user_table limit 5""").show()

+------+---+------+----------+-------+
|userid|age|gender|occupation|zipcode|
+------+---+------+----------+-------+
|   472| 24|     M|   student|  87544|
|   473| 29|     M|   student|  94708|
|   474| 51|     M| executive|  93711|
|   475| 30|     M|programmer|  75230|
|   476| 28|     M|   student|  60440|
+------+---+------+----------+-------+



In [18]:
sqlContext.sql("""SELECT age-userid FROM user_table limit 5""").show()

+--------------+
|(age - userid)|
+--------------+
|            23|
|            51|
|            20|
|            20|
|            28|
+--------------+



In [19]:
sqlContext.sql("""SELECT age-2016 FROM user_table limit 5""").toPandas().rename(columns={"(age - CAST(2016 AS BIGINT))":"2016"})

,2016
0,-1992
1,-1963
2,-1993
3,-1992
4,-1983


In [20]:
sqlContext.sql("""SELECT a.age,count(b.age) FROM user_table a,user_table b 
                    WHERE a.age > b.age
                    AND a.userid < b.userid
                    group by a.age
                    ORDER by a.age""").show()

+---+----------+
|age|count(age)|
+---+----------+
| 11|         1|
| 13|         2|
| 14|         7|
| 15|        41|
| 16|        52|
| 17|       127|
| 18|       272|
| 19|       754|
| 20|      1322|
| 21|      1487|
| 22|      2319|
| 23|      2798|
| 24|      3623|
| 25|      4208|
| 26|      5117|
| 27|      4655|
| 28|      5804|
| 29|      6117|
| 30|      7682|
| 31|      5121|
+---+----------+
only showing top 20 rows



In [21]:
sqlContext.sql("""SELECT a.userid,a.age FROM user_table a,user_table b """).show()

+------+---+
|userid|age|
+------+---+
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
|     1| 24|
+------+---+
only showing top 20 rows



In [22]:
sqlContext.sql(""" SELECT a.userid,a.age, sum(c.age)/(SELECT sum(b.age) FROM user_table as b )
                    FROM user_table a ,user_table c 
                    WHERE a.age <= c.age OR (a.age=c.age and a.userid = c.userid ) 
                    GROUP BY a.age , a.userid 
                    ORDER BY a.age DESC, a.userid ;
                        """).show()

+------+---+-------------------------------------------------------------+
|userid|age|(CAST(sum(age) AS DOUBLE) / CAST(scalarsubquery() AS DOUBLE))|
+------+---+-------------------------------------------------------------+
|   481| 73|                                         0.002273364267696428|
|   767| 70|                                         0.008813179284357386|
|   803| 70|                                         0.008813179284357386|
|   860| 70|                                         0.008813179284357386|
|   559| 69|                                         0.013110772009591728|
|   585| 69|                                         0.013110772009591728|
|   349| 68|                                         0.017346080782286444|
|   573| 68|                                         0.017346080782286444|
|   211| 66|                                         0.019401451216094174|
|   318| 65|                                          0.02547413658870792|
|   564| 65|             

In [23]:
sqlContext.sql(""" SELECT a.userid,a.age, sum(c.age)/(SELECT sum(b.age) FROM user_table as b )
                    FROM user_table a ,user_table c 
                    WHERE a.age <= c.age OR (a.age=c.age and a.userid = c.userid ) 
                    GROUP BY a.age , a.userid 
                    ORDER BY a.age DESC, a.userid ;
                        """).show()

+------+---+-------------------------------------------------------------+
|userid|age|(CAST(sum(age) AS DOUBLE) / CAST(scalarsubquery() AS DOUBLE))|
+------+---+-------------------------------------------------------------+
|   481| 73|                                         0.002273364267696428|
|   767| 70|                                         0.008813179284357386|
|   803| 70|                                         0.008813179284357386|
|   860| 70|                                         0.008813179284357386|
|   559| 69|                                         0.013110772009591728|
|   585| 69|                                         0.013110772009591728|
|   349| 68|                                         0.017346080782286444|
|   573| 68|                                         0.017346080782286444|
|   211| 66|                                         0.019401451216094174|
|   318| 65|                                          0.02547413658870792|
|   564| 65|             

In [24]:
sqlContext.sql("""SELECT a.userid,a.age FROM user_table a,user_table b 
                    where a.age <= b.age OR (a.age=b.age AND a.userid = b.userid)
                    GROUP BY a.userid, a.age
                    ORDER BY a.age DESC, a.userid DESC""").show()

+------+---+
|userid|age|
+------+---+
|   481| 73|
|   860| 70|
|   803| 70|
|   767| 70|
|   585| 69|
|   559| 69|
|   573| 68|
|   349| 68|
|   211| 66|
|   651| 65|
|   564| 65|
|   318| 65|
|   845| 64|
|   423| 64|
|   858| 63|
|   777| 63|
|   364| 63|
|   520| 62|
|   266| 62|
|   934| 61|
+------+---+
only showing top 20 rows



In [25]:
sqlContext.sql("""SELECT a.userid,a.age FROM user_table a,user_table b 
                    where a.age <= b.age OR (a.age=b.age AND a.userid = b.userid)
                    GROUP BY a.userid, a.age
                    ORDER BY a.age DESC, a.userid DESC""").show()

+------+---+
|userid|age|
+------+---+
|   481| 73|
|   860| 70|
|   803| 70|
|   767| 70|
|   585| 69|
|   559| 69|
|   573| 68|
|   349| 68|
|   211| 66|
|   651| 65|
|   564| 65|
|   318| 65|
|   845| 64|
|   423| 64|
|   858| 63|
|   777| 63|
|   364| 63|
|   520| 62|
|   266| 62|
|   934| 61|
+------+---+
only showing top 20 rows



In [26]:
sqlContext.sql("""SELECT a.userid,a.age,b.userid,b.age FROM user_table a,user_table b 
                    where a.gender!=b.gender 
                    AND a.age > b.age
                    AND a.userid != b.userid""").show()

+------+---+------+---+
|userid|age|userid|age|
+------+---+------+---+
|     1| 24|    24| 21|
|     1| 24|    35| 20|
|     1| 24|    36| 19|
|     1| 24|    49| 23|
|     1| 24|    52| 18|
|     1| 24|   150| 20|
|     1| 24|   159| 23|
|     1| 24|   165| 20|
|     1| 24|   180| 22|
|     1| 24|   198| 21|
|     1| 24|   206| 14|
|     1| 24|   223| 19|
|     1| 24|   228| 21|
|     1| 24|   240| 23|
|     1| 24|   258| 19|
|     1| 24|   262| 19|
|     1| 24|   270| 18|
|     1| 24|   274| 20|
|     1| 24|   281| 15|
|     1| 24|   304| 22|
+------+---+------+---+
only showing top 20 rows

